<a href="https://colab.research.google.com/github/djeannie29/Crie-um-agente-de-IA-para-preencher-formul-rios-a-partir-de-seus-documentos-privados/blob/main/Crie_um_agente_de_IA_para_preencher_formul_rios_a_partir_de_seus_documentos_privados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
      )  (
     (   ) )
      ) ( (
    _______)_
 .-'---------|
( C|/\/\/\/\/|
 '-./\/\/\/\/|
   '_________'
    '-------'

# **Estes códigos não funcionam fora de ambientes da IBM.**:

---
Necessárias bibliotecas disponíveis apenas na Cloud da IBM.

**# Bibliotecas e Inicialização do LLM**

---

### **Estágio** **1. Importando bibliotecas**

Por que: Este estágio envolve a preparação do seu ambiente Python com as bibliotecas e configurações necessárias. Isso é essencial para acessar os serviços de IA da IBM Watson e outras funções necessárias no projeto.

Como:

Importe Bibliotecas: Importe bibliotecas como ibm_watson_machine_learning para integração de modelos de IA e BeautifulSoup para análise de conteúdo HTML. 😊

In [2]:
#------------> Import required libraries from various modules. <-------------
# Import required libraries from various modules.
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models import Model

# langchain library for embeddings, text splitting, and conversational retrieval.
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts.prompt import PromptTemplate

# Document loader and vector store modules for processing PDFs.
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.vectorstores import FAISS

# BeautifulSoup for parsing HTML content.
from bs4 import BeautifulSoup

# Flask for web server and CORS for cross-origin resource sharing.
from flask import Flask, jsonify, render_template
from flask_cors import CORS




#












ModuleNotFoundError: No module named 'ibm_watson_machine_learning'

**Estágio 2. Inicializando o modelo de IA**

**Por que:**
Este passo configura e inicializa o modelo de IA que será usado posteriormente para gerar respostas para preencher o formulário. Alternativamente, você pode usar modelos OpenAI GPT ou modelos HuggingFace.

**Como:**

1. **Definir parâmetros:**
   - Defina parâmetros como `MAX_NEW_TOKENS` e `TEMPERATURE`, que controlam o comportamento de geração de texto da IA.

2. **Inicializar o modelo:**
   - Configure a classe `WatsonxLLM` usando o ID do modelo fornecido, credenciais e os parâmetros especificados.
   - A autenticação já foi prearranjada pela equipe da Skills Network, portanto, não é necessário criar uma chave de API.
   - Se você quiser explorar mais o que a plataforma Watsonx tem a oferecer, considere visitar o site oficial para explorar opções de teste gratuito.



In [ ]:

#------------> Function to initialize and retrieve the language model from IBM Watson. <------------
def get_llm():
    # Credentials for accessing IBM Watson services.
    my_credentials = {
        "url": "https://us-south.ml.cloud.ibm.com",
    }

    # Parameters for controlling the generation of text by the model.
    params = {
        GenParams.MAX_NEW_TOKENS: 256, # Maximum tokens generated per run.
        GenParams.TEMPERATURE: 0.0,    # Controls randomness in generation.
    }

    # Initialize the model with specified ID and credentials.
    LLAMA2_model = Model(
        model_id='meta-llama/llama-2-70b-chat',
        credentials=my_credentials,
        params=params,
        project_id="skills-network")

    # Create and return the Watson Language Model.
    llm = WatsonxLLM(model=LLAMA2_model)
    return llm


**Vamos testar!**
---

Testar o seu código regularmente é essencial. Você pode fazer isso executando o script e obtendo uma resposta da instância do LLM. Use o método `.predict` para inserir o seu prompt e, em seguida, exiba o resultado. Por exemplo, tente `llm.predict("Diga-me como é viver em Toronto?")` para ver o que o modelo diz.

Como exercício, experimente a sua instância do LLM com um prompt de sua escolha e imprima a saída. 😊

# Testando a nossa instância do LLM executando-a.
llm = get_llm()
llm_output = llm.predict("Tell how it is like to live in Toronto?")
print(llm_output)

In [ ]:
    # Create and return the Watson Language Model.
    llm = WatsonxLLM(model=LLAMA2_model)
    return llm
    # tesing our llm instance by running it
#llm = get_llm()
#llm_output = llm.predict("Tell how it is like to live in Toronto?")
#print(llm_output)


**Processamento de dados: Estágio 3. Processamento de dados e criação de banco de dados de vetores**

**Por que é necessário:**

Para lidar eficientemente com grandes quantidades de dados de texto e realizar buscas, especialmente em aplicações de processamento de linguagem natural como o preenchimento automático de formulários com IA, é essencial criar um banco de dados de vetores e usar um mecanismo de classificação (ou recuperação).

**Banco de dados de vetores:**

- **Propósito:** Um banco de dados de vetores armazena informações em um formato que um computador pode processar rapidamente. Ele transforma o texto em vetores matemáticos, que são listas de números que representam o significado do texto.
- **Por que é necessário:** Quando lidamos com muitos documentos ou textos extensos, encontrar a informação específica necessária para preencher um formulário pode ser como procurar uma agulha no palheiro. Um banco de dados de vetores permite que a IA pesquise esses "palheiros" incrivelmente rápido para encontrar a "agulha" exata - o pedaço de informação necessário.
- **Benefícios:** Essa abordagem é muito mais rápida do que ler todos os documentos toda vez que você precisa encontrar algo. Além disso, é mais precisa porque a IA entende o contexto da informação que está buscando, graças às propriedades matemáticas dos vetores. Neste projeto de preenchimento automático de formulários, o banco de dados de vetores ajuda a IA a entender o que está lendo nos documentos, e o mecanismo de classificação garante que as informações usadas para preencher os formulários sejam o mais relevantes possível para as perguntas feitas por cada campo do formulário.

**Processamento dos dados envolve:**

1. **Carregamento dos documentos:**
   - Os arquivos PDF do diretório "info" são carregados no sistema. O `PyPDFDirectoryLoader` é usado para carregar arquivos PDF de um diretório.

2. **Divisão do texto:**
   - O texto desses documentos é dividido em partes menores, ou "pedaços", para facilitar o processamento. O `RecursiveCharacterTextSplitter` divide os documentos em pedaços menores para processamento mais fácil.

3. **Criação de embeddings:**
   - A linha `HuggingFaceInstructEmbeddings` refere-se ao uso de um modelo pré-treinado da Hugging Face (uma biblioteca popular de processamento de linguagem natural) para criar embeddings.
   - O modelo específico, "sentence-transformers/all-MiniLM-L6-v2", é projetado para converter sentenças em embeddings. Isso significa que ele lê os pedaços de texto e os transforma em vetores. Cada vetor captura o significado semântico do pedaço de texto, incluindo contexto e a relação das palavras dentro desse pedaço.

4. **Indexação para recuperação:**
   - A linha `FAISS.from_documents` trata de pegar esses vetores e colocá-los em um banco de dados chamado FAISS (Facebook AI Similarity Search).
   - O FAISS é projetado para pesquisar eficientemente esses vetores e encontrar as melhores correspondências para qualquer consulta dada. Isso torna muito mais rápido encontrar pedaços relevantes de texto posteriormente ao preencher formulários.

**Criar um embedding é o processo de transformar pedaços de texto de arquivos PDF em vetores que capturam seu significado, permitindo que a IA pesquise e recupere informações rapidamente.**

Teste

In [ ]:
#------------> Function to process and index PDF documents. <------------
def process_data():
    # Load PDF documents from a specified directory.
    loader = PyPDFDirectoryLoader("info")
    docs = loader.load()

    print("\ntype of docs:",type(docs))
    print("\n docs:", docs)

    # Split the text from documents for better processing.
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=50)
    texts = text_splitter.split_documents(docs)

    print("\ntype of texts:",type(texts))
    print("\ntexts:",texts)

    # Create embeddings for the text data.
    embeddings = HuggingFaceInstructEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    print("\ntype of embeddings:",type(embeddings))
    print("\nembeddings:",embeddings)

    # Index the documents using FAISS for efficient retrieval.
    db = FAISS.from_documents(texts, embeddings)

    print("\ntype of vector db:",type(db))

    return db

process_data()  # this line is only for testing, remove this line before going to the next step

## **rodar comando**  

---
python3.11 Auto_filler_AI.py

**Obtendo os campos do formulário: Estágio 4. Identificação dos campos do formulário**

**Por que:** Para automatizar o preenchimento de formulários, você primeiro precisa saber quais campos estão presentes no formulário e suas características.

**Como:**

1. **Analisar o HTML:**
   - Leia o arquivo HTML e use o BeautifulSoup para analisá-lo.
   - Extraia os campos do formulário: Identifique elementos do formulário, como caixas de entrada, menus suspensos e áreas de texto, e extraia seus rótulos e IDs para identificação.
   - Neste caso, a imagem a seguir mostra um formulário de imposto simples, que é um arquivo HTML dentro da pasta "templates".

2. **A função get_form_field_descriptions foi projetada para ler um arquivo HTML e extrair informações sobre os campos do formulário presentes nele. Aqui está uma explicação passo a passo:**

   - **Abrindo e lendo o arquivo HTML:**
     - A função recebe um caminho de arquivo (html_file_path) como entrada.
     - Ela abre o arquivo HTML especificado e lê seu conteúdo em uma variável chamada html_content.

   - **Analisando o conteúdo HTML:**
     - O conteúdo do arquivo HTML é então analisado usando o BeautifulSoup, uma biblioteca Python para extrair dados de arquivos HTML e XML.
     - Esse processo de análise transforma o conteúdo HTML em um formato que pode ser facilmente navegado e pesquisado programaticamente.

   - **Encontrando os campos do formulário:**
     - A função busca no HTML analisado pelos campos do formulário.
     - Especificamente, ela procura por elementos HTML como input, select e textarea, que são comuns em formulários da web.
     - Cada um desses elementos representa um tipo diferente de campo de entrada em um formulário.

   - **Extraindo informações dos campos:**
     - Para cada campo do formulário encontrado, a função coleta informações relevantes.
     - Primeiro, ela tenta encontrar um elemento de rótulo associado ao campo, que normalmente fornece uma descrição legível pelo ser humano do propósito do campo (como "Nome" ou "Endereço de e-mail").
     - Se um rótulo não for encontrado, a função procura por um atributo de placeholder ou name no campo como alternativa para usar como rótulo.
     - Ela também captura o atributo ID ou name do campo, que geralmente é usado para identificar exclusivamente o campo no formulário.

   - **Armazenando os dados dos campos:**
     - As informações coletadas (rótulo e ID ou name) para cada campo são armazenadas em um dicionário (field_data).
     - Esse dicionário é então adicionado a uma lista (field_info), que contém os dados de todos os campos encontrados no formulário.

   - **Retornando os dados:**
     - Finalmente, a função retorna a lista field_info, que agora contém dicionários com pares de rótulo e ID/nome para cada campo do formulário no arquivo HTML.


In [ ]:
#------------> Function to extract form field descriptions from an HTML file. <------------
def get_form_field_descriptions(html_file_path):
    with open(html_file_path, 'r') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    # Find and process all form fields in the HTML.
    form_fields = soup.find_all(['input', 'select', 'textarea'])
    field_info = []
    for field in form_fields:
        field_data = {}

        # Extract label text or use placeholder/name as a fallback.
        label = soup.find('label', {'for': field.get('id')})
        if label:
            field_data['label'] = label.get_text().strip().rstrip(':')
        else:
            placeholder = field.get('placeholder')
            name = field.get('name')
            description = placeholder if placeholder else name
            if description:
                field_data['label'] = description.strip()

        # Include the ID or name of the field in the data.
        field_id = field.get('id') or field.get('name')
        if field_id:
            field_data['id'] = field_id

        # Add complete field data to the list.
        if 'label' in field_data and 'id' in field_data:
            field_info.append(field_data)

    return field_info


**Tarefa: Acesse a pasta "templates" e abra o arquivo HTML localizado lá. Examine cuidadosamente o formulário e seus vários campos. Considere quais itens dentro do formulário você gostaria de extrair.**





In [ ]:
#teste
def get_form_field_descriptions(html_file_path):
    ...
    return field_info

simple_tax_fields_info = get_form_field_descriptions("templates/styled_tax_form.html")
print(simple_tax_fields_info)

# **Gerando respostas com base nos documentos e nos campos HTML**

## **Estágio 5. Gerando respostas**

Por que: Nesta etapa, o modelo de IA é usado para preencher automaticamente o formulário com base no conteúdo dos documentos carregados.

Antes de entrar no código, vou explicar brevemente a geração com recuperação (RAG).

**Classificação/geração com recuperação:**

Propósito: O mecanismo de recuperação ajuda a decidir quais informações são mais relevantes para a pergunta ou prompt com o qual você está lidando.

Por que é necessário: Quando a IA pesquisa o banco de dados de vetores, ela pode encontrar muitas informações que poderiam ser a resposta. O algoritmo de classificação ordena esses resultados para mostrar a melhor correspondência no topo, assim como os mecanismos de busca mostram os resultados mais relevantes primeiro.

Benefícios: Isso garante que a IA preencha o formulário com as informações mais precisas disponíveis. Economiza tempo, reduzindo a necessidade de verificar manualmente se a IA escolheu os detalhes corretos.

A função filling_form neste script Python foi projetada para preencher automaticamente os campos do formulário usando informações extraídas de documentos. A seguir, uma explicação detalhada de como ela funciona:

Inicialização do modelo de linguagem e processamento de dados: A função começa inicializando um modelo de linguagem (llm) usando a função get_llm(). Esse modelo é capaz de entender e gerar linguagem natural. Ela também chama process_data(), que provavelmente processa documentos (como arquivos PDF) para torná-los pesquisáveis e extrai informações relevantes para uso.
Preparação para o preenchimento do formulário: A função recebe form_fields_info, uma lista de dicionários onde cada dicionário contém informações sobre um campo do formulário (como rótulo e ID). Uma lista vazia chamada structured_responses é criada para armazenar as respostas para cada campo do formulário.
Criação de prompts para cada campo: Para cada campo em form_fields_info, a função gera um prompt. Esse prompt é uma pergunta para o modelo de linguagem, pedindo que ele forneça as informações necessárias para aquele campo específico do formulário. O prompt é criado usando o rótulo e o ID do campo para especificar que informações são necessárias.
Configuração de uma cadeia de recuperação conversacional: Uma cadeia de recuperação conversacional é configurada usando o modelo de linguagem (llm) e o banco de dados de documentos (db). Essa cadeia lida com a tarefa de encontrar a resposta correta nos documentos processados com base no prompt. db.as_retriever provavelmente converte o banco de dados de documentos em um formato que pode ser usado para recuperar informações específicas. O parâmetro search_kwargs={'k': 4} sugere que os 4 melhores resultados mais relevantes sejam recuperados para consideração.
Recuperação e armazenamento de respostas: Para cada campo, a cadeia conversacional é executada com o prompt fornecido. Esse processo envolve o modelo de IA pesquisando o banco de dados de documentos para encontrar a resposta mais relevante. A resposta é extraída do resultado, removendo qualquer espaço em branco extra, e adicionada a structured_responses junto com as informações do campo.
Retornando os campos preenchidos: Após processar todos os campos do formulário, a função retorna structured_responses, que agora contém as informações originais do campo e as respostas correspondentes do modelo de IA.

In [ ]:
#------------> Function to automate form filling using the processed data. <------------
def filling_form(form_fields_info):
    # Initialize the language model and data processing tools.
    llm = get_llm()
    db = process_data()

    structured_responses = []
    for field in form_fields_info:
        # Create a specific prompt for each form field.
        prompt = f"Based on the document, what is the '{field['label']}'? Provide only the required information for the field ID '{field['id']}'."

        # Set up a conversational chain to retrieve and generate responses.
        conversation_chain = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=db.as_retriever(search_kwargs={'k': 4}),
            condense_question_prompt=PromptTemplate(input_variables=[], template=prompt),
        )

        # Get the response for each field.
        result = conversation_chain({"question": prompt, "chat_history": []})
        structured_responses.append({**field, "response": result['answer'].strip()})

    return structured_responses

**Exercício: Teste a função e avalie a saída. Para executá-la, você deve primeiro obter os dados para os campos do formulário (estágio anterior) e passá-los para…**

In [ ]:
#teste
filling_form(form_fields_info):
    ...
    return structured_responses

data_from_form = get_form_field_descriptions("templates/styled_tax_form.html")
structured_responses = filling_form(data_from_form)
print(structured_responses)

Feel free to play with the prompt template and change it to make it more accurate output.

You should get a response like the folowing code. This is a standard way to represent structured data in JSON format, making it suitable for various applications like web APIs, configuration files, and data storage. Theis information is passed to JavaScript to complete the HTML form.

# **Final step: put the mechanism into server and deploy it**
### **Why do you need flask?**

Etapa final: colocar o mecanismo em um servidor e implantá-lo

Por que você precisa do Flask?

Para permitir que o seu código JavaScript complete dinamicamente um formulário usando informações de um LLM (Large Language Model), você deve criar um componente intermediário do lado do servidor, como um aplicativo construído com Flask ou Django. Essa aplicação do lado do servidor atua como uma ponte entre o seu JavaScript do lado do cliente e o LLM baseado em Python. As informações a seguir explicam como você pode estruturar essa configuração:

**Aplicação do lado do servidor (Python):**

Desenvolva uma aplicação do lado do servidor usando um framework web como o Flask. Essa aplicação é executada em um servidor e é responsável por lidar com as solicitações HTTP. Sua aplicação Python se comunica com o LLM para processar dados e gerar respostas.
Criação de um ponto de extremidade da API:
Dentro da sua aplicação do lado do servidor, configure um ponto de extremidade da API. Esse é um URL específico onde o seu servidor aguarda solicitações vindas do JavaScript do lado do cliente. Por exemplo, você pode criar um ponto de extremidade como /api/get-form-data dedicado ao processamento de solicitações relacionadas ao preenchimento de formulários.

Interação com JavaScript do lado do cliente:
O seu código JavaScript, executado no navegador do usuário, faz solicitações para o ponto de extremidade da API do lado do servidor quando precisa de dados para preencher o formulário. O código JavaScript está disponível na pasta “static”. Isso pode ser feito usando a API fetch do JavaScript ou outras bibliotecas de cliente HTTP para enviar solicitações para o seu servidor.

Processamento de solicitações com o LLM:
Quando a aplicação Python do lado do servidor recebe uma solicitação no ponto de extremidade da API, ela interage com o LLM para processar a solicitação. Isso pode envolver o uso do LLM para interpretar a solicitação, buscar os dados necessários ou gerar o texto apropriado para os campos do formulário.

Envio de dados de volta para o JavaScript:
Após processar a solicitação, a aplicação do lado do servidor envia uma resposta de volta para o código JavaScript. Essa resposta contém os dados ou informações necessárias para completar o formulário. A resposta geralmente é formatada como JSON, o que facilita o processamento pelo JavaScript e a atualização do conteúdo da página da web.

Preenchimento de formulários em JavaScript:
Após receber a resposta do servidor, o código JavaScript usa esses dados para preencher dinamicamente os campos do formulário na página da web. Em resumo, você configura uma aplicação do servidor baseada em Python que se comunica com o LLM. Essa aplicação do servidor expõe um ponto de extremidade da API que o seu JavaScript do lado do cliente pode chamar para solicitar dados para o preenchimento de formulários. Ao receber os dados do servidor, o JavaScript preenche os campos do formulário conforme necessário.

# **Etapa 6: Implantação do servidor web**

Por que: O servidor web atua como uma interface entre o usuário e o preenchimento automático de formulários com IA, permitindo que os usuários acessem a função por meio de chamadas à API.

Como:

Configurar o Flask: Crie um aplicativo Flask e configure o CORS para permitir solicitações de origens cruzadas.
Criar um ponto de extremidade da API: Defina uma rota no Flask que é chamada para obter os dados preenchidos do formulário.
Iniciar o servidor: Inicie o servidor Flask para tornar a API acessível.

In [ ]:
#------------> Initialize the Flask application for the web server. <------------
app = Flask(__name__)
CORS(app)  # Enable cross-origin requests.

# Define route for the home page.
@app.route('/')
def home():
    return render_template('styled_tax_form.html')

# Define API route to retrieve form data.
@app.route('/api/get_tax_form_data', methods=['GET'])
def get_tax_form_data():
    data_from_form = get_form_field_descriptions("templates/styled_tax_form.html")
    structured_responses = filling_form(data_from_form)

    # Convert responses to a JSON format for the frontend.
    response_data = {field['id']: field['response'] for field in structured_responses}
    return jsonify(response_data)

# Run the Flask application if this script is executed directly.
if __name__ == '__main__':
    app.run(debug=True, port=5055)

Because you have configured the server to use port=5055, the output message indicates that the server is operational on the default path at port 5055. While the server is running, I aim to launch the web application. To facilitate this, the Skills Network team has provided a plug-in. Follow the provided instructions to access and use the app effectively.